In [1]:
class Action:
    pre = {} #preconditions
    eff={} #effects
    cost=0
    name = ""
    def __init__(self, p, e, c, n):
        self.pre = p
        self.eff = e
        self.name = n
        self.cost = c
        
    def formatFluents(self, d):
        string = ""
        for k in d.keys():
            if(d[k]==0):
                string += "\n- " + k
            else:
                string += "\n+ " + k
        return string

    def __str__(self):
        return f'Name: {self.name} \nCost: {self.cost} \nPreconditions: {self.formatFluents(self.pre)} \nEffects: {self.formatFluents(self.eff)}'

In [2]:
# Get only positive fluents
def getPositiveFluents(state):
    return {key:val for key, val in state.items() if val == 1}


# Get only negative fluents
def getNegativeFluents(state):
    return {key:val for key, val in state.items() if val == 0}


# Check if two states match (match positive fluents only)
def statesMatch(s1,s2):
    s1_pos = getPositiveFluents(s1).copy()
    s2_pos = getPositiveFluents(s2).copy()
    return s1_pos == s2_pos


# Checks if state is a subset of the target – logic rule for checking goal state and applicable actions
def stateSubset(state, target):
    positives = getPositiveFluents(state).copy()
    targetPos = getPositiveFluents(target).copy()
    targetNeg = getNegativeFluents(target).copy()
    subsetPos = targetPos.items() <= positives.items()
    subsetNeg = any(p in positives for p in targetNeg) == False
    return subsetPos & subsetNeg


# Checks if a state fulfills the requirements for a goal state
def atGoal(state, goal):
    remainders = getPositiveFluents(state).copy().items() - getPositiveFluents(goal).copy().items()
    worldCompliment = remainders - getNegativeFluents(goal).copy().items() == set()
    return stateSubset(state, goal) & worldCompliment


# Checks if an action is applicable for a given state
def isApplicable(state, action):
    return stateSubset(state, action.pre)
    

# For a state and all possible actions, return the [applicable actions] for the given state
def applicableActions(state, actions):
    arr = []
    for a in actions:
        if isApplicable(state, a):
            arr.append(a)
    return arr


# Given a state and an action, applies the action and returns the following state if all preconditions are met
def applyAction(state, action):    
    if isApplicable(state, action) == False:
        raise Exception(f'Action {action.name} cannot be applied to state {str(state)}')
    newState = state.copy()
    effects = action.eff.copy()
    for k in effects.keys():
        newState[k] = effects[k]
        if effects[k] == 0:
            newState.pop(k)
    return newState

In [3]:
# Node class for frontier
class Node:
    
    def __init__(self, state, cost, path):
        self.state = state
        self.cost = cost
        self.path = path

    def __str__(self):
        pathNames = [p.name for p in self.path]
        pathStr = '[' + ', '.join(pathNames) + ']'
        return f'State: {self.state}, Cost: {self.cost}, Path: {pathStr}'

In [4]:
def travelHeuristic(nextState):
    atTerminal = False
    atHome = False
    dic = nextState.copy()
    key, value = 'atTerminal',1
    if key in dic and value == dic[key]:
        atTerminal = True
    key, value = 'atHome',1
    if key in dic and value == dic[key]:
        atHome = True

    if atHome == False and atTerminal == False:
        return 1
    if atHome == True and atTerminal == False:
        return 4
    if atHome == False and atTerminal == True:
        return 2
    return 10 # This should never happen

In [5]:
def aStar(start, goal, actions):

    # Init start node, frontier and visited list
    init = Node(start, 0, [])
    frontier = [init]
    visited = [init]

    while True:
        
        # Get first node from frontier
        head = frontier.pop(0)

        state = head.state.copy()

        # When A* has found a path, return it
        if (atGoal(state, goal)):
            print(f'A* found a path with cost: {head.cost}')
            return head.path

        applicActions = applicableActions(state, actions)
        
        for action in applicActions:
            nextState = applyAction(state, action).copy()
            newPath = head.path + [action]
            cost = travelHeuristic(nextState) + head.cost + action.cost
            newNode = Node(nextState, cost, newPath)

            # Only apply action if we haven't visited the state before
            vis = False
            for i in range(len(visited)):
                if visited[i].state == nextState:
                    vis = True
                    if cost < visited[i].cost:
                        visited[i] = newNode
                        frontier.append(newNode)
            if vis == False:
                visited.append(newNode)
                frontier.append(newNode)

        # Sort frontier based on costs
        frontier = sorted(frontier, key=lambda x:x.cost)
        


# ORIGINAL CASE – NO WEIGHTS CHANGED 

In [6]:
demoGoal = {'atDestination':1,'haveBags':1}
demoStart = {"atHome":1} 

demoA1 = Action(p={"atHome":1},e={"haveBags":1},c=1,n="Pack")
demoA2 = Action(p={"haveBags":1, "carFueled":0},e={"carFueled":1},c=2,n="FuelCar")
demoA3 = Action(p={"haveBags":1,"carFueled":1,"atDestination":0},e={"carFueled":0,'atDestination':1,"atHome":0},c=5,n="Drive")
demoA4 = Action(p={"haveBags":1, "atHome":1},e={"atTerminal":1,"atHome":0},c=2,n="Go To Terminal")
demoA5 = Action(p={"atTerminal":1,"haveBags":1,"boarded":0},e={"boarded":1},c=1,n="Board")
demoA6 = Action(p={"atTerminal":1, "boarded":0, "checkedIn":0},e={"checkedIn":1},c=1,n="Check In")
demoA7 = Action(p={"checkedIn":0,"boarded":1, "atDestination":0},e={"atDestination":1, "atTerminal":0, "boarded":0},c=3,n="Take Train")
demoA8 = Action(p={"checkedIn":1,"boarded":1, "atDestination":0},e={"atDestination":1,"atTerminal":0,"boarded":0, "checkedIn":0},c=1,n="Take Plane")

demoActions=[demoA1,demoA2,demoA3,demoA4,demoA5,demoA6,demoA7,demoA8]

solution = aStar(demoStart, demoGoal, demoActions)
print(f'Solution: {[a.name for a in solution]}')

A* found a path with cost: 16
Solution: ['Pack', 'Go To Terminal', 'Board', 'Take Train']


# NEXT CASE – ADD COST + 2 to TAKE TRAIN, SHOULD CHANGE THE PATH TO EITHER 
## ['Pack', 'Go To Terminal', ‘Check in’, ’Board', 'Take Plane’] = 17 (cost + h), or
## ['Pack', 'Fuel', ‘Drive’] = 17 (cost + h)

In [7]:
demoA7 = Action(p={"checkedIn":0,"boarded":1, "atDestination":0},e={"atDestination":1, "atTerminal":0, "boarded":0},c=5,n="Take Train")

demoActions=[demoA1,demoA2,demoA3,demoA4,demoA5,demoA6,demoA7,demoA8]

solution = aStar(demoStart, demoGoal, demoActions)
print(f'Solution: {[a.name for a in solution]}')

A* found a path with cost: 17
Solution: ['Pack', 'FuelCar', 'Drive']


# NEXT CASE – ADD COST + 1 TO DRIVE, SHOULD RETURN 
## ['Pack', 'Go To Terminal', ‘Check in’, ’Board', 'Take Plane’] = 17 (cost + h)

In [8]:
demoA3 = Action(p={"haveBags":1,"carFueled":1,"atDestination":0},e={"carFueled":0,'atDestination':1,"atHome":0},c=6,n="Drive")


demoActions=[demoA1,demoA2,demoA3,demoA4,demoA5,demoA6,demoA7,demoA8]

solution = aStar(demoStart, demoGoal, demoActions)
print(f'Solution: {[a.name for a in solution]}')

A* found a path with cost: 17
Solution: ['Pack', 'Go To Terminal', 'Check In', 'Board', 'Take Plane']
